<a href="https://colab.research.google.com/github/cookie13v/Ruhi-DataScience/blob/main/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df = pd.read_csv('/content/loan_data.csv')
display(df.head())

,age,years_employed,annual_income,credit_score,credit_history_years,savings_assets,current_debt,defaults_on_file,delinquencies_last_2yrs,derogatory_marks,loan_amount,interest_rate,debt_to_income_ratio,loan_to_income_ratio,payment_to_income_ratio,loan_status
0,40,17.2,25579,692,5.3,895,10820,0,0,0,600,17.02,0.423,0.023,0.008,1
1,33,7.3,43087,627,3.5,169,16550,0,1,0,53300,14.10,0.384,1.237,0.412,0
2,42,1.1,20840,689,8.4,17,7852,0,0,0,2100,18.33,0.377,0.101,0.034,1
3,53,0.5,29147,692,9.8,1480,11603,0,1,0,2900,18.74,0.398,0.099,0.033,1
4,32,12.5,63657,630,7.2,209,12424,0,0,0,99600,13.92,0.195,1.565,0.522,1


In [4]:
from sklearn.preprocessing import MinMaxScaler

# Instantiate MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to X and transform X into scaled data
X_scaled_array = scaler.fit_transform(X)

# Convert the scaled array back to a DataFrame, preserving column names
X_scaled = pd.DataFrame(X_scaled_array, columns=X.columns)

print("X_scaled DataFrame head:")
display(X_scaled.head())

X_scaled DataFrame head:


,age,years_employed,annual_income,credit_score,credit_history_years,savings_assets,current_debt,defaults_on_file,delinquencies_last_2yrs,derogatory_marks,loan_amount,interest_rate,debt_to_income_ratio,loan_to_income_ratio,payment_to_income_ratio
0,0.423077,0.431078,0.045017,0.685259,0.176667,0.002983,0.065897,0.0,0.000000,0.0,0.001005,0.648235,0.527569,0.007526,0.007530
1,0.288462,0.182957,0.119519,0.555777,0.116667,0.000563,0.100990,0.0,0.111111,0.0,0.530653,0.476471,0.478697,0.616658,0.615964
2,0.461538,0.027569,0.024851,0.679283,0.280000,0.000057,0.047721,0.0,0.000000,0.0,0.016080,0.725294,0.469925,0.046663,0.046687
3,0.673077,0.012531,0.060200,0.685259,0.326667,0.004933,0.070693,0.0,0.111111,0.0,0.024121,0.749412,0.496241,0.045660,0.045181
4,0.269231,0.313283,0.207051,0.561753,0.240000,0.000697,0.075721,0.0,0.000000,0.0,0.995980,0.465882,0.241855,0.781234,0.781627


In [5]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (40000, 15)
Shape of X_test: (10000, 15)
Shape of y_train: (40000,)
Shape of y_test: (10000,)


In [6]:
from sklearn.tree import DecisionTreeClassifier as DTC

# create the model
DTC_algo = DTC()
DTC_algo

DecisionTreeClassifier()

In [8]:
DTC_model = DTC_algo.fit(X_train, y_train)
DTC_model

DecisionTreeClassifier()

In [12]:
from sklearn.metrics import accuracy_score

# Make predictions on the test data
y_pred = DTC_model.predict(X_test)

# Create a DataFrame for comparison
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

print("First 5 Actual vs. Predicted values:")
display(results_df.head())

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

First 5 Actual vs. Predicted values:


,Actual,Predicted
33553,1,1
9427,1,1
199,0,0
12447,0,0
39489,1,1



Model Accuracy: 0.8407


In [13]:
from sklearn.metrics import f1_score, recall_score, precision_score

f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

print(f"F1 Score: {f1:.4f}")
print(f"Recall Score: {recall:.4f}")
print(f"Precision Score: {precision:.4f}")

F1 Score: 0.8549
Recall Score: 0.8549
Precision Score: 0.8550


In [18]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# this is the list of parameters we will tune. Some are fixed values some are distributions
tuned_parameters = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(3, 9),  # Draw from a uniform distribution between 3 and 15
    'min_samples_split': randint(3, 9),  # Draw from a uniform distribution between 2 and 10
    'max_features': ['sqrt', 'log2', None]
}

# we will tune against f1_macro (to find the best choice)
score = 'f1_macro' # arbitrary choice

print(f"Tuning hyperparameters for {score}")
print("\n")

# do the search using 5 folds/chunks
clf = RandomizedSearchCV(DTC(), tuned_parameters, cv=5, random_state=1984,
                         scoring= score, n_iter=20, refit=True)

# pass the data to fit/train
clf.fit(X_train, y_train)

print("Best parameters set found on the training set:")
print(clf.best_params_)

Tuning hyperparameters for f1_macro


Best parameters set found on the training set:
{'criterion': 'entropy', 'max_depth': 8, 'max_features': None, 'min_samples_split': 7}


In [20]:
DTC_algo = DTC(**clf.best_params_) # get the best parameters from grid search
DTC_model = DTC_algo.fit(X_train, y_train)

In [21]:
from sklearn.metrics import accuracy_score

# Make predictions on the test data
y_pred = DTC_model.predict(X_test)

# Create a DataFrame for comparison
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

print("First 5 Actual vs. Predicted values:")
display(results_df.head())

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

First 5 Actual vs. Predicted values:


,Actual,Predicted
33553,1,1
9427,1,1
199,0,0
12447,0,0
39489,1,1



Model Accuracy: 0.8648


In [14]:
from sklearn.linear_model import LogisticRegression as LogR

# create the model
logR_algo = LogR(penalty='l2')
logR_algo


LogisticRegression()

In [15]:
logR_model = logR_algo.fit(X_train, y_train)
logR_model

LogisticRegression()

In [17]:
from sklearn.metrics import accuracy_score

# predict the test data
predict = logR_model.predict(X_test)

# Create a DataFrame for comparison
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

print("First 5 Actual vs. Predicted values:")
display(results_df.head())

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

First 5 Actual vs. Predicted values:


,Actual,Predicted
33553,1,1
9427,1,1
199,0,0
12447,0,0
39489,1,1



Model Accuracy: 0.8407
